<a href="https://colab.research.google.com/github/conorgibbons147/cyclegan-map/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/conorgibbons147/cyclegan-map.git

fatal: destination path 'cyclegan-map' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append('/content/cyclegan-map/models')

In [5]:
! ls /content/cyclegan-map/models

discriminator.py  generator.py


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import itertools
from generator import Generator
from discriminator import Discriminator
from dataset import ImageDataset
from utils import weights_init_normal, ReplayBuffer, sample_images

SyntaxError: invalid syntax (generator.py, line 44)